# 常用资料

- Google Earth Engine homepage: https://earthengine.google.com/
- API reference: https://developers.google.com/earth-engine/apidocs/
- Coding tips: https://developers.google.com/earth-engine/guides/
- Official datasets: https://developers.google.cn/earth-engine/datasets/
- Community datasets that aren't included in the official catalogue: https://gee-community-catalog.org/projects/

# 登入Google

In [3]:
# 调用所有GEE算法
import ee
# 调用地图可视化
import geemap.core as geemap

In [4]:
# 启动身份验证
ee.Authenticate()

# 根据项目初始化，此处换成你注册的项目名称
ee.Initialize(project='ee-ivywpsonwork')

# 调用数据集

In [ ]:
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate(
    '2021-05-01', '2021-06-01'
)


# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )


dataset = dataset.map(apply_scale_factors)

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

m = geemap.Map()
m.set_center(116.4074, 39.9042, 8)
m.add_layer(dataset, visualization, 'True Color (432)')
m

Map(center=[39.9042, 116.4074], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

# 使用filter筛选研究区域

In [ ]:
dataset = (
  ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
  # 筛选影像获取日期
  .filterDate('2021-05-01', '2021-06-01')
  # 筛选影像空间范围
  .filterBounds(ee.Geometry.Rectangle([110,35,125,45]))
  # 筛选影像属性
  .filter(ee.Filter.lt('CLOUD_COVER', 10))
)


# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )


dataset = dataset.map(apply_scale_factors).median()

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

m = geemap.Map()
m.set_center(116.4074, 39.9042, 8)
m.add_layer(dataset, visualization, 'True Color (432)')
m

Map(center=[39.9042, 116.4074], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

# 上传数据

在Gee Web Code Editor中，左侧工具栏找到Assets，单击New上传数据。可以是tif，shp或csv  
**不调用脚本的话，只能手动逐个操作**

# 组合多个数据集

In [15]:
# 设定影像时间范围
start_date = '2021-01-01'
end_date = '2021-12-31'

# 设置影像空间范围，大致是北京周边
beijing_bounds = ee.Geometry.Rectangle(
    [115.4, 39.4, 117.6, 41.6]
)

# 加载Landsat-8影像并筛选云量<10%的图像
landsat8 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate(start_date, end_date)
    .filterBounds(beijing_bounds)
    .filter(ee.Filter.lt('CLOUD_COVER', 10))
)

# 将栅格值映射至0-1范围
# 乘数和加数请参见Data Catalog中Band选项卡的Scale和Offset两项
def apply_landsat_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

landsat8 = landsat8.map(apply_landsat_scale_factors)

# 加载Sentinel-1影像并筛选对应属性
sentinel1 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterDate(start_date, end_date)
    .filterBounds(beijing_bounds)
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    .filter(ee.Filter.eq('resolution_meters', 10))
    .select('VV', 'VH')
)

# 加载Sentinel-2影像并筛选云量<10%的图像
sentinel2 = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate(start_date, end_date)
    .filterBounds(beijing_bounds)
    .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 10))
)

# 将栅格值映射至0-1范围
# 乘数和加数请参见Data Catalog中Band选项卡的Scale和Offset两项
def apply_sentinel2_scale_factors(image):
    optical_bands = image.select('B.*').multiply(0.0001)
    return image.addBands(optical_bands, None, True)

sentinel2 = sentinel2.map(apply_sentinel2_scale_factors)

# 你上传的文件路径
beijing_area = ee.Image('projects/ee-ivywpsonwork/assets/rasterized_shp/beijing')

# 打印筛选范围内遥感影像的数量
print('Landsat 8 collection:', landsat8.size().getInfo())
print('Sentinel-1 collection:', sentinel1.size().getInfo())
print('Sentinel-2 collection:', sentinel2.size().getInfo())
print('Beijing area asset:', beijing_area.getInfo())

# 拼接多波段影像
fusion_bands = beijing_area.addBands(landsat8).addBands(sentinel1).addBands(sentinel2)

# 试图展示图像时，代码报错
'''
EEException: Image.addBands, argument 'srcImg': Invalid type.
Expected type: Image<unknown bands>.
Actual type: ImageCollection.
'''
m = geemap.Map()
m.set_center(116.4074, 39.9042, 8)
m.add_layer(fusion_bands, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.3})
m.add_layer(fusion_bands, {'bands': ['b1'],'palette': 'red'}, 'Beijing Area')
m

Landsat 8 collection: 71
Sentinel-1 collection: 122
Sentinel-2 collection: 459
Beijing area asset: {'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -128, 'max': 127}, 'dimensions': [4560, 4010], 'crs': 'EPSG:3857', 'crs_transform': [10, 0, 12930926.81572864, 0, -10, 4871941.222768805]}], 'version': 1729496550570340.0, 'id': 'projects/ee-ivywpsonwork/assets/rasterized_shp/beijing', 'properties': {'system:footprint': {'type': 'LinearRing', 'coordinates': [[116.16042297313176, 40.04125715696923], [116.16041936720333, 40.04124232442971], [116.1604198179444, 39.764965226320186], [116.16046741619536, 39.764923794793376], [116.1605104168802, 39.76487525588861], [116.26292235515373, 39.7648752334083], [116.36530780694925, 39.76487528507025], [116.46769330818371, 39.76487528719163], [116.57007871005166, 39.7648752487877], [116.57013270624674, 39.764911826445186], [116.57019588177884, 39.76494482914512], [116.57019633174696, 40.04123480689444

EEException: Image.addBands, argument 'srcImg': Invalid type.
Expected type: Image<unknown bands>.
Actual type: ImageCollection.

## ImageCollection与Image

In [14]:
'''
ImageCollection是一组图片
Image是单个图片
大部分图像处理操作只能在Image上进行
'''

landsat8_median = landsat8.median()
sentinel1_median = sentinel1.median()
sentinel2_median = sentinel2.median()

print('Type of Landsat8_median:', type(landsat8_median))
print('Type of Sentinel1_median:', type(sentinel1_median))
print('Type of Sentinel2_median:', type(sentinel2_median))

# 拼接多波段影像
fusion_bands_median = beijing_area.addBands(landsat8_median).addBands(sentinel1_median).addBands(sentinel2_median)

m = geemap.Map()
m.set_center(116.4074, 39.9042, 8)
m.add_layer(fusion_bands_median, {'bands': ['SR_B4', 'SR_B3', 'B2'], 'min': 0.0, 'max': 0.3}, 'Fusion Landsat-8/Sentinel-2 Median')
m

Type of Landsat8_median: <class 'ee.image.Image'>
Type of Sentinel1_median: <class 'ee.image.Image'>
Type of Sentinel2_median: <class 'ee.image.Image'>


Map(center=[39.9042, 116.4074], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

其他常用的将imageCollection压缩为image的方法包括：  
- median(): 中位数
- mean(): 平均值
- min(): 最小值
- max(): 最大值
- sum(): 求和
- count(): 每个像素对应的image数量
- mode(): 众数
- reduce(ee.Reducer.sth()): 调用其他Reducer执行操作
- mosaic(): 按图像叠盖的先后顺序自动镶嵌多张影像  
更多方法可参见API Reference

# 更多图像处理操作

## 掩膜去云

In [26]:
def mask_s2_clouds(image):
  qa = image.select('QA60')

  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Sentinel-2中QA60波段的第10位和第11位标记了该像素是否被云覆盖
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  # 将未被云覆盖的区域作为掩膜
  return image.updateMask(mask).divide(10000)

# 设置影像空间范围，大致是北京周边
beijing_bounds = ee.Geometry.Rectangle(
    [115.4, 39.4, 117.6, 41.6]
)

dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-07-01', '2020-07-30')
    .filterBounds(beijing_bounds)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
)

dataset_cloudless = dataset.map(mask_s2_clouds)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(116.4074, 39.9042, 8)
m.add_layer(dataset_cloudless.mean(), visualization, 'RGB')
m

Map(center=[39.9042, 116.4074], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

去云后的图像如果有空洞，说明选择的遥感影像数量不够，无云的影像没法填补有云影像空出来的区域。此时可以调整选择标准，如加长时间区间，或提高云分数。  
一般来说，如果想获取更清晰的影像，median的效果比mean更好。  

## 计算NDVI

In [27]:
start_date = '2024-06-01'
end_date = '2024-08-31'

beijing_bounds = ee.Geometry.Rectangle(
    [115.4, 39.4, 117.6, 41.6]
)

landsat8 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate(start_date, end_date)
    .filterBounds(beijing_bounds)
    .filter(ee.Filter.lt('CLOUD_COVER', 10))
)

def apply_landsat_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

landsat8 = landsat8.map(apply_landsat_scale_factors)

# NDVI = (NIR - Red) / (NIR + Red)
# 对于Landsat-8，SR_B4是红光波段, SR_B5是近红外波段
def calculate_ndvi(image):
  return image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

ndvi_collection = landsat8.map(calculate_ndvi)
ndvi_median = ndvi_collection.median()

ndvi_vis = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['white', 'green']
}

m = geemap.Map()
m.set_center(116.4074, 39.9042, 8)
m.add_layer(ndvi_median, ndvi_vis, 'Landsat 8 NDVI')
m

Map(center=[39.9042, 116.4074], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

# 下载影像

In [32]:
start_date = '2023-01-01'
end_date = '2023-12-31'

beijing_bounds = ee.Geometry.Rectangle(
    [116.375, 39.875, 116.5, 40]
)

embedding = (
    ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
    .filterDate(start_date, end_date)
    .filterBounds(beijing_bounds)
    .median()
)

task_config = {
      'description': 'Satellite_Embedding_V1_Beijing',      # 下载下来的影像名称
      'folder': 'Tutorial',                   # 下载到的Google Drive文件夹
      'scale': 30,                        # 空间分辨率
      'region': beijing_bounds,                 # 影像区域
      'maxPixels': 1e13                     # 最大像元数量
  }
task = ee.batch.Export.image.toDrive(embedding,**task_config)
task.start()

embedding_vis = {'min': -0.3, 'max': 0.3, 'bands': ['A01', 'A16', 'A09']};
m = geemap.Map()
m.set_center(116.4074, 39.9042, 8)
m.add_layer(embedding, embedding_vis, 'Satellite Embedding Beijing')
m

Map(center=[39.9042, 116.4074], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## 可能出现的各种问题

- ***EEException: User memory limit exceeded.***: 你下载的文件处理起来过于麻烦，GEE分配给你的存储不够用了。
- ***EEException: Exported bands must have the same projection.***: 你下载的多源影像坐标系不同，需要在task_config中显式指定CRS，或者使用reproject()方法提前完成坐标系转换
- ***EEException: Image.addBands, argument 'srcImg': Invalid type. Expected type: Image<unknown bands>. Actual type: ImageCollection.***: 你试图输出ImageCollection，请转换为Image
- ***Error: Exported bands must have compatible data types; found inconsistent types: [Type1] and [Type2]***: 你下载的多源影像波段之间的数据类型不同，请使用.cast方法将波段转成同一数据类型
- ***EEException: Computation timed out.***: 你执行的图像操作过于复杂，请简化操作或缩小执行区域，控制EECU时间不要太大。

# 机器学习

## 采样

In [38]:
beijing_bounds = ee.Geometry.Rectangle(
    [115.4, 39.4, 117.6, 41.6]
)

landsat8 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate('2021-06-01', '2021-08-31')
    .filterBounds(beijing_bounds)
    .filter(ee.Filter.lt('CLOUD_COVER', 10))
    .median()
)

def apply_landsat_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

landsat8 = apply_landsat_scale_factors(landsat8)

training_points = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(116.328, 39.945), {'landcover': 0}), # 城市(label 0)
    ee.Feature(ee.Geometry.Point(116.440, 39.922), {'landcover': 0}),
    ee.Feature(ee.Geometry.Point(116.436, 39.997), {'landcover': 1}), # 植被(label 1)
    ee.Feature(ee.Geometry.Point(116.193, 40.004), {'landcover': 1}),
    ee.Feature(ee.Geometry.Point(116.267, 39.991), {'landcover': 2}), # 水体(label 2)
    ee.Feature(ee.Geometry.Point(116.381, 39.927), {'landcover': 2}),
])

# Print the image and training points to inspect (optional)
print('Landsat 8 composite:', landsat8.getInfo())
print('Training points:', training_points.getInfo())

# 解释变量波段名称
feature_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']
image_features = landsat8.select(feature_bands)

# 采样
sampled_data = image_features.sampleRegions(**{
    'collection': training_points,
    'properties': ['landcover'], # 作为label的属性名称
    'scale': 30 # 采样空间分辨率
})

print('Sampled data:', sampled_data.getInfo())

m = geemap.Map()
m.set_center(116.4074, 39.9042, 8)
m.add_layer(landsat8, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.3}, 'Landsat 8 Composite')
m.add_layer(training_points, {'color': 'red'}, 'Training Points')
m

Landsat 8 composite: {'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.2, 'max': 1.6022125}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.2, 'max': 1.6022125}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.2, 'max': 1.6022125}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.2, 'max': 1.6022125}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.2, 'max': 1.6022125}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'SR_B6', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -0.2, 'max': 1.6022125}, 'crs': 'EPSG:4326', 'crs_

Map(center=[39.9042, 116.4074], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## 预测

In [39]:
# 初始化随机森林分类器
classifier = ee.Classifier.smileRandomForest(numberOfTrees=50)

# 训练分类器
trained_classifier = classifier.train(**{
    'features': sampled_data,
    'classProperty': 'landcover',
    'inputProperties': feature_bands
})

# 对整个区域进行分类
classified_image = image_features.classify(trained_classifier)

# 可视化
classification_vis = {
    'min': 0,
    'max': 2,
    'palette': ['808080', '008000', '0000FF']
}

m.add_layer(classified_image, classification_vis, 'Landsat 8 Classification')
m

Map(bottom=198819.0, center=[39.98185552901966, 116.33010864257814], controls=(ZoomControl(options=['position'…

# 经验之谈

1. 代码执行程序已断开连接：一般等一会儿就好了。但是如果长时间放置不管，程序还是有可能自行中断。
2. User memory limit exceeded/Computation timed out：简化程序或者氪金。如何简化程序请参考https://developers.google.com/earth-engine/guides/debugging
3. 访问受限：Google说什么就是什么，不要抬杠
4. 下下来的影像后面有一个特别长的后缀名：影像太大，GEE自动切割了，手动拼一下就行。
5. 由于一些地区原因，有的时候氪金都氪不了。这时候可以淘宝/闲鱼搜搜有没有提供相关服务的。或者，可以尝试申请免费配额提升。这个比较苛刻，要求你的研究主题和可持续发展紧密相关，而且申请之后会等很久才下结果，无论是否批准提升。https://developers.google.cn/earth-engine/help?hl=zh-cn#additional_quota